# I Construction de la base de données

## Régression classique

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import ruptures as rpt

data_folder = 'datas/'

# Charger le DataFrame à partir du fichier CSV
final_data = pd.read_csv(os.path.join(data_folder, 'final_data.csv'), parse_dates=['DATE'])

# Ajouter les colonnes à retirer à la liste selected_columns
selected_columns = ['BOP', 'business_insolvencies', 'firms_creation', 'natality_rate', 'unemployment_rate', 'Valuation', 'Exch_rate', 'Fixed rate', 'negotiable_debts']

# Garder uniquement les colonnes spécifiées dans selected_columns
df = final_data[selected_columns]
# Sélectionner les colonnes nécessaires
data_for_regression = final_data[selected_columns]

data_for_regression = data_for_regression.dropna()

# Retirer les colonnes spécifiées de X
X = data_for_regression.drop(columns=['negotiable_debts'])
Y = data_for_regression['negotiable_debts']
X = sm.add_constant(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialiser le modèle de régression linéaire avec statsmodels
model = sm.OLS(Y_train, X_train).fit()

# Imprimer un résumé des statistiques du modèle
print(model.summary())

# Faire des prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Évaluer la performance du modèle
mse = mean_squared_error(Y_test, predictions)
r2 = r2_score(Y_test, predictions)

print(f'\nMean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

matrix = pd.concat([Y, X], axis=1).to_numpy()


                            OLS Regression Results                            
Dep. Variable:       negotiable_debts   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                     535.9
Date:                Sat, 30 Dec 2023   Prob (F-statistic):           1.69e-94
Time:                        17:24:07   Log-Likelihood:                -1692.4
No. Observations:                 137   AIC:                             3403.
Df Residuals:                     128   BIC:                             3429.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   1.63e+

## Détection de grandes périodes et points de rupture

Nous venons donc de réaliser une régression linéaire classique afin de trouver des corrélations, significatives ou non, entre nos régresseurs et l'objet d'étude (la dette négociable de l'Etat). Cependant, il semble essentiel de souligner que ces corrélations peuvent changer au cours du temps. L'économie n'est pas un champ d'étude atemporel, certains coefficients devant des régresseurs peuvent positifs et significatifs pendant une certaine période, par exemple après la crise de 2008, puis perdre en significativité voire changer de signe pendant une période plus calme. Nous avons donc eu l'idée de trouver des grandes périodes de l'économie entre 2009 et 2023. Ces périodes se trouvent en cherchant des "points de rupture", c'est à dire des instants où les coefficients devant les régresseurs tendent à changer. 

In [2]:
# Détection de changement avec ruptures pour chaque coefficient
breakpoints = []
for i in range(X.shape[1]):
    algo = rpt.Pelt(model="rbf").fit(matrix[:, [0, i + 1]])
    result = algo.predict(pen=10)  # Vous pouvez ajuster le paramètre de pénalité
    breakpoints.append(result)

# Convertissez les indices des points de rupture en un seul ensemble
breakpoints = np.unique(np.concatenate(breakpoints))

# Affichez les points de rupture
print("Points de rupture détectés :", breakpoints)

# Réinitialiser l'index du dataframe
final_data = final_data.reset_index(drop=True)

# On écrit alors les points de rupture détectés
breakpoints = [0, 40, 85, 135, len(final_data) - 1]

Points de rupture détectés : [ 40  85 135 172]


Nous avons utilisé l'algorithme Pelt de la bibliothèque "ruptures" afin de détecter les points de rupture. Ce modèle Pelt utilise le modèle "rbf" (radial basis function) pour estimer le coût d'ajustement entre segments successifs de la série temporelle des données économiques. L'algorithme fonctionne de la manière suivante : on initie une segmentation triviale, puis on propage et ajuste itérativement cette segmentation tout en écartant les branches non prometteuses. La détection de ruptures repose sur la minimisation du coût global de la segmentation. L'Nous avons utilisé ce modèle car il permet d'identifier des périodes où les coefficients de la régression linéaire présentent des changements significatifs, facilitant ainsi l'analyse des influences temporelles sur la dette de l'État.

Ainsi, nous trouvons 4 périodes. La première démarre en janvier 2009 et se termine en mai 2012, la seconde démarre en mai 2012 et se termine en février 2016. La troisième commence en février 2016 et s'achève en avril 2020. La dernière commence en avril 2020 et se termine en 2023, lors de nos dernières données. 

Une fois ces périodes trouvées, on peut alors reproduire une régression linéaire pour chaque segment afin de visualiser, commenter et interpréter les coefficients de chaque segment. 

In [3]:

# Extraire les dates correspondantes aux points de rupture
breakpoint_dates = final_data.loc[breakpoints, 'DATE']

# Ajustez les dates pour le premier et le dernier segment
breakpoint_dates.iloc[0] = pd.to_datetime('2009-01-01')
breakpoint_dates.iloc[-1] = pd.to_datetime('2023-04-01')


# Diviser le dataframe en segments basés sur les dates de rupture
segments = []
for i in range(len(breakpoints) - 1):
    start_date = pd.to_datetime(breakpoint_dates.iloc[i])
    end_date = pd.to_datetime(breakpoint_dates.iloc[i + 1])
    segment_data = final_data[(final_data['DATE'] >= start_date) & (final_data['DATE'] < end_date)]
    segments.append(segment_data)


# Ajuster une régression linéaire pour chaque segment avec statsmodels
for i, segment_data in enumerate(segments):
    start_date = pd.to_datetime(breakpoint_dates.iloc[i])
    end_date = pd.to_datetime(breakpoint_dates.iloc[i + 1])

    X = segment_data.drop(columns=['negotiable_debts', 'DATE', 'Deposit facility', 'Marginal lending facility'])
    Y = segment_data['negotiable_debts']

    # Ajoutez une constante à X pour estimer l'ordonnée à l'origine
    X = sm.add_constant(X)

    # Créer un modèle de régression linéaire avec statsmodels
    model = sm.OLS(Y, X)

    # Ajuster le modèle
    results = model.fit()

    # Afficher les résultats sous forme de tableau
    print(f"\nSegment {i + 1} - Period: {start_date} to {end_date}")
    print(results.summary())



Segment 1 - Period: 2009-01-01 00:00:00 to 2012-05-01 00:00:00


                            OLS Regression Results                            
Dep. Variable:       negotiable_debts   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                     188.2
Date:                Sat, 30 Dec 2023   Prob (F-statistic):           4.46e-24
Time:                        17:24:07   Log-Likelihood:                -434.93
No. Observations:                  40   AIC:                             887.9
Df Residuals:                      31   BIC:                             903.1
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -9.602e+

C:\Users\gabri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\regression\linear_model.py:1966: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


La première période (2009-avril 2012) est une période post-crise 2008. Comme attendu et commenté plus haut, le coefficient devant la colonne "Balance of payments", "unemployment_rate", "Valuation" sont significatifs et positif. Les coefficients devant "firms_creation", "natality_rate", "Exch_rate" et "Fixed rate" sont non significatifs sûrement dû au faible nombre de données sur cette période de 3 ans. Le coefficients devant 'business_insolvencies" est négatif mais assez faible, tout comme dans la régression globale. Test